In [1]:
import robotic as ry
from utility import *

In [2]:
PUZZLE_NAME = "p1-two-blocks"
# PUZZLE_NAME = "p2-maze-easy"
# PUZZLE_NAME = "p3-maze"
# PUZZLE_NAME = "p4-four-blocks"
# PUZZLE_NAME = "p5-wall-easy"
# PUZZLE_NAME = "p6-wall"
# PUZZLE_NAME = "p7-o-room"
# PUZZLE_NAME = "p8-corner"
# PUZZLE_NAME = "p9-cube-free"

In [3]:
config = ry.Config()
config.addFile(f"puzzles/{PUZZLE_NAME}.g")

EGO_NAME = "ego"
GOAL_OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"
object_names = []
for frame_name in config.getFrameNames():
    attributes = config.getFrame(frame_name).getAttributes()
    if "logical" in attributes and "object" in attributes["logical"]:
        object_names.append(frame_name)

object_names.sort()

GAP_COEFFICIENT = 1.1

config.view()
f = config.getFrame(CAMERA_NAME)
config.view_setCamera(f)

q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(GOAL_OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [4]:
if len(config.getCollisions()) > 0:
    print("Initial configuration is in collision")
    exit()

In [5]:
def solve_touch_with_komo(komo_config, obj_name):
    komo = ry.KOMO(komo_config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e2])
    komo.addObjective([], ry.FS.negDistance, [EGO_NAME, obj_name], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [6]:
ry.params_add({
    "rrt/stepsize": 0.05
})
def solve(cur_config : ry.Config, object_name: str, goal_name : str):
    q_begin = cur_config.getJointState()
    q_ego = cur_config.getFrame(EGO_NAME).getPosition()[:2]
    q_object = cur_config.getFrame(object_name).getPosition()[:2]
    q_target = cur_config.getFrame(goal_name).getPosition()[:2]

    ITERATIONS = 16
    delta_theta = 2 * np.pi / ITERATIONS
    theta_offset = np.random.uniform(0, 2 * np.pi)
    touch_positions = []
    gap = cur_config.getFrame(EGO_NAME).getSize()[1] + max(cur_config.getFrame(object_name).getSize()) * GAP_COEFFICIENT
    for i in range(ITERATIONS):
        transpose_polar1 = np.array([gap, i * delta_theta + theta_offset])
        transpose_cartesian1 = np.array(polar_to_cartesian(transpose_polar1))
        q_home1 = q_object + transpose_cartesian1 - q_ego + q_begin
        cur_config.setJointState(q_home1)
        # view config
        # cur_config.view_setCamera(cur_config.getFrame(CAMERA_NAME))
        # cur_config.view(True)

        q_sub = solve_touch_with_komo(cur_config, object_name)
        if q_sub is not None:
            touch_positions.append(q_sub)

    # reshuffle touch positions
    np.random.shuffle(touch_positions)

    for q_sub in touch_positions:
        cur_config.setJointState(q_begin)
        copy_config = ry.Config()
        copy_config.addConfigurationCopy(cur_config)
        copy_config.setJointState(q_sub)
        if copy_config.getCollisionsTotalPenetration() > 0:
            continue

        rrt = ry.PathFinder()
        rrt.setProblem(cur_config, [q_begin], [q_sub])
        solution1 = rrt.solve()
        del rrt
        if not solution1.feasible:
            continue
        
        cur_config.setJointState(q_sub)
        cur_config.attach(EGO_NAME, object_name)
        q_real_goal = q_target + q_sub - q_object - q_ego + q_begin

        rrt = ry.PathFinder()
        rrt.setProblem(cur_config, [q_sub], [q_real_goal])
        solution2 = rrt.solve()
        del rrt
        cur_config.attach(FLOOR_NAME, object_name)
        cur_config.setJointState(q_begin)
        if solution1.feasible and solution2.feasible:
            return solution1, solution2
        
    return None, None

In [7]:
# forward subproblem search algorithm
def forward_subproblem_search(start_config : ry.Config):
    start_node = SolutionNode(start_config, GOAL_OBJ_NAME)
    L = SolutionTree(start_node)
    
    view_config = ry.Config()

    while L.get_len() > 0:
        cur_node = L.get_next_node() # get the next best one
        x = cur_node.get_config()

        view_config.clear()
        view_config.addConfigurationCopy(x)
        view_config.view_setCamera(view_config.getFrame(CAMERA_NAME))
        view_config.view(message=f"""node_depth: {cur_node.depth}
tree_depth: {L.get_depth()}
tree_mode: {L.mode}""")

        sol1, sol2 = solve(x, GOAL_OBJ_NAME, GOAL_NAME)
        if sol1 is not None:
            del view_config
            return cur_node.get_path()
        
        for object_name in object_names:
            if not reachable(x, object_name):
                continue
            subgoals = propose_subgoals(x, object_name)
            display_points(x, subgoals, target=object_name) # for testing purposes
            for subgoal in subgoals:
                new_config = ry.Config()
                new_config.addConfigurationCopy(x)
                new_config.getFrame(SUB_GOAL_NAME).setPosition(subgoal.tolist() + [0.2])
                if reject(new_config, L):
                    # new_config.view_setCamera(new_config.getFrame(CAMERA_NAME))
                    # new_config.view(True, message="REJECTED")
                    # new_config.view_close()
                    del new_config
                    continue

                path1, path2 = solve(new_config, object_name, SUB_GOAL_NAME)
                if path1 is not None:
                    new_config.setJointState(path1.x[-1])
                    new_config.attach(EGO_NAME, object_name)
                    new_config.setJointState(path2.x[-1])
                    new_config.attach(FLOOR_NAME, object_name)
                    new_node = cur_node.add_child(new_config, object_name)
                    L.add_node(new_node)
                else:
                    del new_config
    del view_config
    return None

In [8]:
path = forward_subproblem_search(config)

if path is not None:
    print("Solution found!")

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_front) [9,5] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
proxy:  (ego)-(outwall_front) [9,5] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego

imDefLkup.c,430: The application disposed a key event with 394 serial.


-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_back) [9,3] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_right) [9,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_right) [9,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_right) [9,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_right) [9,2] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (ego)-(outwall_

In [9]:
save_pngs = False
for i in range(1, len(path)):
    next_config = path[i].get_config()
    target_obj_name = path[i].target_obj
    config.getFrame(SUB_GOAL_NAME).setPosition(next_config.getFrame(target_obj_name).getPosition())
    solution1, solution2 = None, None
    while solution1 is None:
        solution1, solution2 = solve(config, target_obj_name, SUB_GOAL_NAME)
            
    animate_solution(config, solution1, save_pngs, png_path=f"temp/{PUZZLE_NAME}-p-{i}-1")
    config.attach(EGO_NAME, target_obj_name)
    animate_solution(config, solution2, save_pngs, png_path=f"temp/{PUZZLE_NAME}-p-{i}-2")
    config.attach(FLOOR_NAME, target_obj_name)

solution1, solution2 = None, None
while solution1 is None:
    solution1, solution2 = solve(config, GOAL_OBJ_NAME, GOAL_NAME)
    
animate_solution(config, solution1, save_pngs, png_path=f"temp/{PUZZLE_NAME}-z-1")
config.attach(EGO_NAME, GOAL_OBJ_NAME)
animate_solution(config, solution2, save_pngs, png_path=f"temp/{PUZZLE_NAME}-z-2")
config.attach(FLOOR_NAME, GOAL_OBJ_NAME)

-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj2)-(ego) [13,9] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
proxy:  (ego)-(outwall_front) [9,5] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (obj1)-(ego) [12,9] 	d=-0
proxy:  (ego)-(outwall_left) [9,4] 	d=-0


In [10]:
config.view_close()
del config